# IMPORTATION DES MODULE NECESSAIRE

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# desactivation des Warning
import warnings
warnings.filterwarnings("ignore")


# Recuperartion du nom des villes avec le code geo

In [ ]:
#lecture file 
meta_data = pd.read_csv("meta_dossier_complet.csv", sep  = ";", encoding = "utf8")

#recuperation des colonnes
cod_geo = meta_data[meta_data['COD_VAR'] == "CODGEO"][["COD_MOD", "LIB_MOD"]]
cod_geo.drop_duplicates(subset = 'COD_MOD', inplace = True)

# enregistrement des donnée recuperer dans des fichier csv
cod_geo.to_csv("codegeo.csv", index = False)

# NETTOYAGE DE DATA (fichier principale)

## lecture du fichier "dossier_complet.csv"

In [3]:
df = pd.read_csv("dossier_complet.csv", sep=";", encoding="utf8", low_memory = False)

df.head()

,CODGEO,P20_POP,P20_POP0014,P20_POP1529,P20_POP3044,P20_POP4559,P20_POP6074,P20_POP7589,P20_POP90P,P20_POPH,...,CPGEO524,VV24,VVUH24,VVLIT24,RT24,RTUH24,RTLIT24,AJCS24,AJCSUH24,AJCSLIT24
0,01001,806.0,155.306150,97.169380,148.761022,191.372049,143.616249,64.277500,5.497650,410.480215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01002,262.0,57.242248,33.095709,56.756395,56.376187,38.184730,18.601713,1.743019,137.369209,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01004,14288.0,2649.807321,3027.794160,2691.337871,2427.065245,2169.634700,1131.402296,190.958407,6880.642235,...,0.0,0.0,0.0,0.0,1.0,16.0,16.0,0.0,0.0,0.0
3,01005,1782.0,353.000000,273.000000,369.000000,387.000000,277.000000,113.000000,10.000000,893.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01006,113.0,9.912281,13.877193,19.824561,23.789474,37.666667,7.929825,0.000000,59.473684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.shape

(35001, 1883)

In [13]:
df[df['CODGEO'] == '13009']

,CODGEO,P20_POP,P20_POP0014,P20_POP1529,P20_POP3044,P20_POP4559,P20_POP6074,P20_POP7589,P20_POP90P,P20_POPH,...,CPGEO524,VV24,VVUH24,VVLIT24,RT24,RTUH24,RTLIT24,AJCS24,AJCSUH24,AJCSLIT24
4297,13009,845.0,135.048598,130.552836,147.660445,248.183115,135.65829,43.7848,4.111915,404.88215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## detection des valeurs vides (colonne et ligne)

### suppression des lignes avec trop  de colonne vide

In [5]:
# nombre de colonne vide par ligne 
n = df.isna().sum(axis=1).sort_values(ascending=False) 
n


35000    1862
34995    1862
34990    1862
34992    1862
34993    1862
         ... 
34924      12
34923      12
34922      12
34894      12
34921      12
Length: 35001, dtype: int64

In [6]:
# on vois qu'il y'a des ligne avec trop de colonne vide, nous allons les supprimer
df = df.drop(df[df.isna().sum(axis=1) >= 63].index)
df.shape
# n = df.isna().sum(axis=1).sort_values(ascending=False) 
# n 

(34946, 1883)

### suppression des colonnes avec trop de ligne vide

In [7]:
# Detection de nombre de celule vide par colonne 
n = df.isna().sum(axis=0).sort_values(ascending=False)
n.head(100)

P14_RP_MIBOIS    34851
P20_RP_MIDUR     34851
P09_RP_BDWC      34851
P09_RP_CHOS      34851
P09_RP_CLIM      34851
                 ...  
D68_RP              25
DECE6875            25
D82_LOG             25
D75_LOG             25
D68_LOG             25
Length: 100, dtype: int64

In [8]:
# on vois qu'il y'a des variable avec trop ligne avec vide, nous allons les supprimer
# recuperation des colonne avec au plus 100 valeur vide
col = df.columns[df.isna().sum(axis=0).sort_values(ascending=True) <= 100]
df = df.loc[:, col]
df.shape

### remplissage de nan restant par 0

(34946, 1829)

In [26]:
n = df['D121'].isna().sum(axis=0)#.sort_values(ascending=False)
n

37

In [10]:
#df = df.fillna(0)

## verification du type des colonnes et transformation si nécessaire

In [11]:
# verification des type des colonnes
df.dtypes.value_counts()

float64    1801
object       28
Name: count, dtype: int64

In [12]:
# recuperation des champs object sans codgeo 
object_columns = list(df.select_dtypes(include=['object']).columns)
object_columns.remove('CODGEO')
df_object = df[object_columns]

df_object.head()

,NBMENFISC21,NBPERSMENFISC21,MED21,PIMP21,TP6021,TP60AGE121,TP60AGE221,TP60AGE321,TP60AGE421,TP60AGE521,...,PPEN21,PPAT21,PPSOC21,PPFAM21,PPMINI21,PPLOGT21,PIMPOT21,D121,D921,RD21
0,346,895,25820,s,s,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
1,115,266,24480,s,s,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
2,6855,15092,21660,"50,0","17,0","17,0","22,0","19,0","16,0","11,0",...,"25,3","10,2","7,0","2,6","3,0","1,4","-14,8",11890,36450,"3,1"
3,800,2028,24610,"61,0",s,s,s,s,s,s,...,"21,4","10,0","3,4","2,1","0,9","0,4","-16,3",15560,40580,"2,6"
4,51,107,24210,s,s,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s


In [13]:
#fonction pour convertir les chaine en nombre
def trans_str_in_float(val):
    """
    essaie de convertir val en float : 
        si ca echoue alors c'est du texte, donc on retourn 0
        sinon on retourne val
    param : 
        val (str or int or float)
    return : 
        val or 0 (foat)
    ex : 
    >>> trans_str_in_float('3.5')
    3.5
    >>> trans_str_in_float('popp')
    0
    """
    try : 
        nb = float(val)
        return nb
    except : 
        return 0

In [14]:
# SUPPRESSION DES VIRGULE DANS LES CHAMPS
# TRANSFORMATION DES NOMBRE EN FLOAT ET DES STR EN 0 
for i in object_columns: 
    try :
        df_object[i] = df_object[i].str.replace(",", ".")
    except :
        pass
        
    df_object[i] = df_object[i].map(trans_str_in_float)



In [15]:
# ENREGISTREMENT DES MODIFICATION DAN df 
df[object_columns] = df_object

In [16]:
# verification des type encore
df.dtypes.value_counts()

float64    1828
object        1
Name: count, dtype: int64

# regroupement de donnée

## creation de nouvelle colonne departement

In [17]:
#creation fonction pour recuperer le departemet
def get_departement(codegeo):
    """
    cette fonction permet de recuperer les deux premier caractere de CODGEO
    et retourn le code du department
    params : 
        codegeo [str] : code de la ville ou commune
    return 
        dep[str] : code du departement associer
    ex : 
    >>>get_departement('1001')
    01
    >>>get_departement('75094')
    75
    """
    codegeo = str(codegeo)
    code = ["0"] * (5 - len(codegeo)) + list(codegeo)
    return "".join(code[0:2])


In [18]:
# fonction pour tester notre fonction get_departement
def test_get_departement():
    assert get_departement('09646') == '09'
    assert get_departement('1646') == '01'
    assert get_departement('75646') == '75'
    print("fonction get_departement : OK !")

# test de notre fonction
test_get_departement()

fonction get_departement : OK !


In [19]:
#parcours de CODGEO pour creer une nouvel colonne 'dep' qui contient le numero du departement
df["DEPARTEMENT"] = df["CODGEO"].map(get_departement)

In [20]:
df.shape

(34946, 1830)

# recuperation des ville avec plus de 1000 habitant en 2020

In [21]:
data  = df.copy()

In [22]:
data = data[data['P20_POP'] >= 1000]
data.to_csv('data_net_5000.csv')